In [1]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam
import numpy as np

Using TensorFlow backend.


# Preparing data

In [2]:
filename = 'input_data.csv'
raw_data = open(filename, 'rt')
data = np.loadtxt(raw_data, delimiter= '\t')

In [3]:
print data.shape

(426088, 1148)


In [4]:
number_of_features = len(data[0])-1
size_of_dataset = len(data)

In [5]:
input_features = data[:,0:number_of_features]
labels = data[:,number_of_features] #last column consists of labels

Now let's split data into trainig and validation set (90/10)

In [6]:
np.random.seed(0) #seed fixed for reproducibility
mask = np.random.rand(size_of_dataset) < 0.9  #array of boolean variables

training_set = input_features[mask]
training_labels = labels[mask]

validation_set = input_features[~mask]
validation_labels = labels[~mask]

# Neural network model

In [22]:
inputs = Input(shape=(number_of_features,))

output_from_1st_layer = Dense(1000, activation='relu')(inputs)
output_from_1st_layer = Dropout(0.2)(output_from_1st_layer)
output_from_1st_layer = BatchNormalization()(output_from_1st_layer)

output_from_2nd_layer = Dense(500, activation='relu')(output_from_1st_layer)
output_from_2nd_layer = Dropout(0.2)(output_from_2nd_layer)
output_from_2nd_layer = BatchNormalization()(output_from_2nd_layer)


output_from_3rd_layer = Dense(300, activation='relu')(output_from_2nd_layer)
output_from_3rd_layer = Dropout(0.2)(output_from_3rd_layer)
output_from_3rd_layer = BatchNormalization()(output_from_3rd_layer)


output = Dense(1, activation='sigmoid')(output_from_3rd_layer)

model = Model(inputs, output)

Model compilation with Adam optimizer

In [25]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Now, let's train the model

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1147)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1148000   
_________________________________________________________________
batch_normalization_4 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_6 (Dense)              (None, 500)               500500    
_________________________________________________________________
batch_normalization_5 (Batch (None, 500)               2000      
_________________________________________________________________
dense_7 (Dense)              (None, 300)               150300    
_________________________________________________________________
batch_normalization_6 (Batch (None, 300)               1200      
__________

In [20]:
model.fit(training_set, training_labels, batch_size=256, nb_epoch=3, 
          validation_data=(validation_set, validation_labels))

/Users/pawelmorawiecki/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 426088 samples, validate on 42445 samples
Epoch 1/3
426088/426088 [==============================] - 169s - loss: 0.4197 - acc: 0.7983 - val_loss: 0.3137 - val_acc: 0.8597
Epoch 2/3
426088/426088 [==============================] - 156s - loss: 0.3150 - acc: 0.8579 - val_loss: 0.2469 - val_acc: 0.8957
Epoch 3/3
426088/426088 [==============================] - 141s - loss: 0.2527 - acc: 0.8893 - val_loss: 0.1841 - val_acc: 0.9251


Saving weights

In [11]:
model.save_weights("model_1147_features.h5")

# Evaluation

In [23]:
model.load_weights("model_1147_features.h5")

In [12]:
filename = 'test_set.csv'
raw_data = open(filename, 'rt')
test_data = np.loadtxt(raw_data, delimiter= '\t')

In [13]:
test_set = test_data[:,0:number_of_features]
test_labels = test_data[:,number_of_features] #last column consists of labels

In [14]:
print test_set.shape

(40764, 1147)


In [26]:
scores = model.evaluate(test_set, test_labels, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

40512/40764 [============================>.] - ETA: 0sacc: 77.27%


and for a single example

In [27]:
single_example = test_set[4:5,:] #example number 5 from the test set
prediction = model.predict(single_example)
print '%.8f' % prediction[0]

0.99831080
